<a href="https://colab.research.google.com/github/alarcon7a/gemini_vs_gpt/blob/main/Gemini_vs_GPT4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Instalando algunas librerias necesarias

In [ ]:
pip install -q -U langchain-google-genai langchain gradio openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.1/803.1 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.4/225.4 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.7/205.7 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 381.9/381.9 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.5 

## Importando librerias

In [ ]:
from getpass import getpass
import os
import gradio as gr
import time

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import HumanMessage, SystemMessage

## Configurando keys

In [ ]:
GOOGLE_API_KEY = getpass('Enter the secret value: ')
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

Enter the secret value: ··········


In [ ]:
OPENAI_API_KEY = getpass('Enter the secret value: ')
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

Enter the secret value: ··········


# Gemini vs GPT4  - Texto

## Google Gemini + Langchain

In [ ]:
llm_gemini = ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.8, max_tokens=2000)
response = llm_gemini.invoke("Quien eres?")

In [ ]:
response.content

'Soy un gran modelo de lenguaje, entrenado por Google.'

## OpenAI GPT-4 + Langchain

In [ ]:
chat = ChatOpenAI( model="gpt-4",temperature=0.8, max_tokens=2000,)

In [ ]:
template = (
    "You are a helpful assistant"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
response = chat(
    chat_prompt.format_prompt(
        text="Quien eres?"
    ).to_messages()
)

In [ ]:
response.content

'Soy un asistente inteligente de inteligencia artificial diseñado para ayudar con la información y realizar tareas.'

## Funciones

In [ ]:
def gemini_response(text):
    start_time = time.time()  # Inicia el contador de tiempo
    response = llm_gemini.invoke(text) # Invoca el modelo
    end_time = time.time()  # Finaliza el contador de tiempo
    execution_time = end_time - start_time
    return response.content, execution_time

In [ ]:
def gpt4_response(text):
    start_time = time.time()  # Inicia el contador de tiempo

    response = chat(
        chat_prompt.format_prompt(
            text=text
        ).to_messages()
    ) # Invoca el modelo
    end_time = time.time()  # Finaliza el contador de tiempo
    execution_time = end_time - start_time
    return response.content, execution_time

In [ ]:
def gemini_vs_gpt4(text_question):
    gemini_list = []
    gpt_list = []
    gemini_content, gemini_execution_time = gemini_response(text_question)
    gpt4_content, gpt4_execution_time = gpt4_response(text_question)

    gemini_time_formatted = f"{gemini_execution_time:.2f} segundos"
    gpt4_time_formatted = f"{gpt4_execution_time:.2f} segundos"

    gemini_list.append((text_question, gemini_content))
    gpt_list.append((text_question, gpt4_content))
    return gemini_list, gemini_time_formatted, gpt_list, gpt4_time_formatted

## Gradio

In [ ]:
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            txt_time_gemini = gr.Textbox(label="Tiempo de ejecución de Gemini Pro", interactive=False)
            chat_response_gemini = gr.Chatbot(label="Respuesta de Gemini")
        with gr.Column():
            txt_time_gpt = gr.Textbox(label="Tiempo de ejecución de GPT-4", interactive=False)
            chat_response_gpt = gr.Chatbot(label="Respuesta de GPT-4",)
    with gr.Row():
        txt_question = gr.Textbox(label="Pregunta...")
        btn_send = gr.Button(value="Enviar")
        btn_send.click(
            gemini_vs_gpt4,
            inputs=txt_question,
            outputs=[chat_response_gemini, txt_time_gemini, chat_response_gpt, txt_time_gpt]
        )

demo.launch(debug=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://299a7325d9574bcff0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


ERROR:tornado.access:500 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 691.81ms
ERROR:tornado.access:500 POST /v1beta/models/gemini-pro:generateContent?%24alt=json%3Benum-encoding%3Dint (127.0.0.1) 584.29ms


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://299a7325d9574bcff0.gradio.live


In [ ]:
demo.close()

Closing server running on port: 7860


# Gemini vs gpt4 - Vision

## Gemini vision

In [ ]:
import google.generativeai as genai
from openai import OpenAI
import base64
import io

In [ ]:
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model_vision = genai.GenerativeModel('gemini-pro-vision')

In [ ]:
def gemini_vision(img, query):
    gemini_list = []
    start_time = time.time()  # Inicia el contador de tiempo

    response = model_vision.generate_content([query, img]) ## Inferencia al modelo de vision
    response.resolve()

    end_time = time.time()  # Finaliza el contador de tiempo
    execution_time = end_time - start_time
    gemini_time_formatted = f"{execution_time:.2f} segundos"

    gemini_list.append((query, response.text)) # Formato de tupla en lista

    return gemini_list, gemini_time_formatted

## GPT4 vision

In [ ]:
client = OpenAI()

In [ ]:
def gpt_vision(img, query):
    gpt_list = []

    buffer = io.BytesIO()
    img.save(buffer, format='JPEG')  # You can change 'JPEG' to the appropriate format of your image
    byte_data = buffer.getvalue()
    # Now encode this bytes object with base64
    encode_image = base64.b64encode(byte_data).decode('utf-8')

    start_time = time.time()  # Inicia el contador de tiempo
    response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": query},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encode_image}"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)
    end_time = time.time()  # Finaliza el contador de tiempo
    execution_time = end_time - start_time
    gemini_time_formatted = f"{execution_time:.2f} segundos"
    gpt_list.append((query, response.choices[0].message.content)) # Formato de tupla en lista

    return gpt_list , gemini_time_formatted

## Todo junto

In [ ]:
def geminiv_vs_gpt4v(img, query):

    gemini_content, gemini_execution_time = gemini_vision(img, query) # Ejecutando Gemini Vision
    gpt4_content, gpt4_execution_time = gpt_vision(img, query)  # Ejecutando GPT 4 Vision

    return gemini_content, gemini_execution_time, gpt4_content, gpt4_execution_time

## Gradio interface - Vision

In [ ]:
with gr.Blocks() as demo_vision:
    with gr.Row():
        with gr.Column():
            txt_time_gemini = gr.Textbox(label="Tiempo de ejecución de Gemini Pro", interactive=False)
            chat_response_gemini = gr.Chatbot(label="Respuesta de Gemini Pro")
        with gr.Column():
            txt_time_gpt = gr.Textbox(label="Tiempo de ejecución de GPT4", interactive=False)
            chat_response_gpt = gr.Chatbot(label="Respuesta de GPT4")
    with gr.Row():
        img_question = gr.Image(label="Pregunta...",type="pil")
        txt_question = gr.Textbox(label="Pregunta...")
    with gr.Row():
        btn_send = gr.Button(value="Enviar")
        btn_send.click(
            geminiv_vs_gpt4v,
            inputs=[img_question,txt_question],
            outputs=[chat_response_gemini, txt_time_gemini,chat_response_gpt,txt_time_gpt]
        )

demo_vision.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://446ac7bbbc9414ba5f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://446ac7bbbc9414ba5f.gradio.live


In [ ]:
demo_vision.close()

Closing server running on port: 7860
